In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext dotenv

In [3]:
import os

In [4]:
!pwd
!ls -l ./

/data/datadrive01/prj/AzureChestXRayNoAML/code
total 568
-rwxrwxrwx 1 loginVM_001 loginVM_001 278518 Oct 17 07:02 00_create_docker_image.html
-rwxrwxrwx 1 loginVM_001 loginVM_001  18633 Oct 17 07:07 00_create_docker_image.ipynb
drwxrwxrwx 3 loginVM_001 loginVM_001   4096 Sep 26 20:16 01_DataPrep
drwxrwxrwx 3 loginVM_001 loginVM_001   4096 Oct  4 15:15 02_Model
drwxrwxr-x 2 loginVM_001 loginVM_001   4096 Oct 17 07:05 docker_history
-rwxrwxrwx 1 loginVM_001 loginVM_001 256265 Sep 26 17:06 edit_python_files.html
-rwxrwxrwx 1 loginVM_001 loginVM_001   6867 Sep 26 17:08 edit_python_files.ipynb
drwxrwxrwx 3 loginVM_001 loginVM_001   4096 Sep 26 17:15 src


In [5]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "/chestxray-no-aml-gpu"
image_version = '1.0.1'

Overwriting .env


In [6]:
%dotenv
dockerfile_name = 'dockerfile'+ '_' + os.getenv('image_version')

In [7]:
%%writefile ./../docker/$dockerfile_name

FROM nvidia/cuda:9.0-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget \
      ssh \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh 
        
#https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
# https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
# https://www.microsoft.com/net/download/linux-package-manager/ubuntu16-04/runtime-2.1.2
RUN wget -q https://packages.microsoft.com/config/ubuntu/16.04/packages-microsoft-prod.deb && \
    dpkg -i packages-microsoft-prod.deb && \ 
    apt-get install  -y --no-install-recommends apt-transport-https && \
    apt-get update && \
    apt-get install  -y --no-install-recommends aspnetcore-runtime-2.1  && \
    mkdir /tmp/azcopy && \
    wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
    tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
    /tmp/azcopy/install.sh && \
    rm -rf /tmp/azcopy

# Install Python packages and keras
ENV NB_USER keras
ENV NB_UID 1000

RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
    chown $NB_USER $CONDA_DIR -R && \
    mkdir -p /src && \
    chown $NB_USER /src

USER $NB_USER

ARG python_version=3.6

# tensorflow-gpu 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      scikit-image \
      tensorflow-gpu \
      tensorflow-tensorboard \
      matplotlib \
      numpy==1.14.5 && \
    conda install \
      bcolz \
      h5py \
      matplotlib \
      mkl \
      nose \
      notebook=5.6.0 \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      requests \
      scikit-learn \
      six \
      tqdm && \
    conda install -c conda-forge imageio && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    pip install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl \
      torchvision && \
    conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile_1.0.1


In [8]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':' + os.getenv('image_version')
docker_file_location = os.path.join(*(['.', '..','docker', dockerfile_name]))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/chestxray-no-aml-gpu:1.0.1'

'./../docker/dockerfile_1.0.1'

In [9]:
!mkdir -p ./docker_history
!cp $docker_file_location ./docker_history/

In [10]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/chestxray-no-aml-gpu:1.0.1 -f ./../docker/dockerfile_1.0.1 .
Sending build context to Docker daemon  4.967MB
Step 1/16 : FROM nvidia/cuda:9.0-cudnn7-devel
 ---> ed1be95f3fce
Step 2/16 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 2725672a79ae
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget       ssh       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> a365ec832596
Step 4/16 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> ac194d98ffc0
Step 5/16 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> f460ccfeb992
Step 6/16 : RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh &&     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64

  Running setup.py bdist_wheel for networkx: started
  Running setup.py bdist_wheel for networkx: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
  Running setup.py bdist_wheel for termcolor: started


  Running setup.py bdist_wheel for termcolor: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for gast: started
  Running setup.py bdist_wheel for gast: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/3c/33/ae/db8cd618e62f87594c13a5483f96e618044f9b01596efd013f
  Running setup.py bdist_wheel for html5lib: started
  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
  Running setup.py bdist_wheel for toolz: started
  Running setup.py bdist_wheel for toolz: finished with 

icu-58.2-h9c2b 100% |###############################| Time: 0:00:00  67.58 MB/s
jpeg-9b-h024ee 100% |###############################| Time: 0:00:00  66.34 MB/s
libglu-9.0.0-h 100% |###############################| Time: 0:00:00  67.10 MB/s
libgpuarray-0. 100% |###############################| Time: 0:00:00  55.32 MB/s
libopus-1.2.1- 100% |###############################| Time: 0:00:00  60.23 MB/s
libsodium-1.0. 100% |###############################| Time: 0:00:00  65.43 MB/s
libuuid-1.0.3- 100% |###############################| Time: 0:00:00  38.58 MB/s
libvpx-1.7.0-h 100% |###############################| Time: 0:00:00  67.86 MB/s
libxcb-1.13-h1 100% |###############################| Time: 0:00:00  63.72 MB/s
lzo-2.10-h49e0 100% |###############################| Time: 0:00:00  59.19 MB/s
mkl-2019.0-118 100% |###############################| Time: 0:00:03  68.23 MB/s
ncurses-6.0-h9 100% |###############################| Time: 0:00:00  67.41 MB/s
pcre-8.42-h439 100% |###################

scikit-learn-0 100% |###############################| Time: 0:00:00  63.43 MB/s
bcolz-1.2.1-py 100% |###############################| Time: 0:00:00  64.71 MB/s
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - imageio


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    imageio-2.3.0              |             py_1         3.1 MB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         7.1 MB

The following NEW packages will be I

  Running setup.py develop for Keras
  Cloning git://github.com/keras-team/keras.git to /tmp/pip-req-build-43mmhbxl
  Running setup.py bdist_wheel for Keras: started
  Running setup.py bdist_wheel for Keras: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-_8emn1j6/wheels/e6/02/ad/5e8e1a5824af71082e2260fe7e2eaa1b745c34706e6ff0a14b
Successfully built Keras
  Cloning https://github.com/aleju/imgaug to /tmp/pip-req-build-tx8i7w3k
  Running setup.py bdist_wheel for imgaug: started
  Running setup.py bdist_wheel for imgaug: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-i44dj3zr/wheels/9c/f6/aa/41dcf2f29cc1de1da4ad840ef5393514bead64ac9e644260ff
Successfully built imgaug
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-wvdi4yp3
  Running setup.py bdist_wheel for keras-contrib: started
  Running setup.py bdist_wheel for keras-contrib: finished with status 'done'
  Stored in directory: /tmp/pip-ephem

libuuid-1.0.3-h1bed415_2.tar.bz2              16 KB
backcall-0.1.0-py36_0.tar.bz2                 19 KB
pygpu-0.7.6-py36h035aef0_0.tar.bz2           702 KB
tqdm-4.26.0-py36h28b3542_0.tar.bz2            59 KB
pycrypto-2.6.1-py36_6.tar.bz2                477 KB
freeglut-3.0.0-hf484d3e_5.tar.bz2            251 KB
wcwidth-0.1.7-py36_0.tar.bz2                  25 KB
requests-2.14.2-py36_0.tar.bz2               720 KB
ruamel_yaml-0.11.14-py36_1.tar.bz2           401 KB
partd-0.3.9-py36_0.tar.bz2                    32 KB
hdf5-1.10.2-hba1933b_1.tar.bz2               5.2 MB
entrypoints-0.2.3-py36_2.tar.bz2               9 KB
gstreamer-1.14.0-hb453b48_1.tar.bz2          3.8 MB
libopencv-3.4.2-hb342d67_1.tar.bz2          40.4 MB
py-opencv-3.4.2-py36hb342d67_1.tar.bz2       1.2 MB
jupyter_core-4.4.0-py36_0.tar.bz2             63 KB
conda-env-2.6.0-1.tar.bz2                      3 KB
olefile-0.46-py36_0.tar.bz2                   48 KB
python-3.6.3-hc9025b9_1.tar.bz2             27.0 MB
libxcb-1.13-

Removing intermediate container 14a842ce9a25
 ---> 17a1940c9e5f
Step 14/16 : ENV PYTHONPATH='/src/:$PYTHONPATH'
 ---> Running in d2717334be51
Removing intermediate container d2717334be51
 ---> 9e187efd8856
Step 15/16 : WORKDIR /src
Removing intermediate container e07578bbb737
 ---> 50ac1d08a844
Step 16/16 : EXPOSE 8888
 ---> Running in 94528f77e6fa
Removing intermediate container 94528f77e6fa
 ---> 0f5bea828c8d
Successfully built 0f5bea828c8d
Successfully tagged georgedockeraccount/chestxray-no-aml-gpu:1.0.1


In [11]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/chestxray-no-aml-gpu]

b16fb501: Preparing 
45a5055d: Preparing 
ccafd4b4: Preparing 
c015134f: Preparing 
827381d4: Preparing 
0a87ad4a: Preparing 
88acbd3d: Preparing 
31a34e1b: Preparing 
6ca4feed: Preparing 
e3905f52: Preparing 
6eeb5f04: Preparing 
9e19929c: Preparing 
b2f378bb: Preparing 
afdbe580: Preparing 
43c86cbc: Preparing 


16fb501: Pushing  1.745GB/5.605GB12APushing  636.7MB/5.605GB

16fb501: Pushing  3.574GB/5.605GB

16fb501: Pushing  5.426GB/5.605GB

16fb501: Pushed   5.693GB/5.605GB1.0.1: digest: sha256:07334a770a555b44fa54f2ac612391623e052d72f47c8876418c002efac7942d size: 3688


In [12]:
!echo nvidia-docker run -i -t -p 10003:8888 -v '$(pwd)':/local_dir:rw $docker_image_name /bin/bash -c '"/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip='*' --port=8888 --no-browser --allow-root"'

nvidia-docker run -i -t -p 10003:8888 -v $(pwd):/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.1 /bin/bash -c "/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip=* --port=8888 --no-browser --allow-root"


In [13]:
!jupyter nbconvert --to html 00_create_docker_image.ipynb

[NbConvertApp] Converting notebook 00_create_docker_image.ipynb to html
[NbConvertApp] Writing 329329 bytes to 00_create_docker_image.html
